In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from urllib.request import urlopen
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
epl20 = pd.read_csv("https://www.football-data.co.uk/mmz4281/1920/E0.csv")
epl21 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/E0.csv")
epl22 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2122/E0.csv")

columns = urlopen("https://www.football-data.co.uk/notes.txt") 

In [3]:
usable = ['HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC']

In [4]:
def form_guide(x, team):
  result = []
  if x.FTR == 'H' and x.HomeTeam == team:
    result.append(3)
  elif x.AwayTeam == team and x.FTR == 'A':
    result.append(3)
  elif x.FTR == 'D':
    result.append(1)
  else:
    result.append(0)
  return sum(result)

def shots(x, team):
  if x.HomeTeam == team:
    return x.HS
  if x.AwayTeam == team:
    return x.AS

def shots_ot(x, team):
  if x.HomeTeam == team:
    return x.HST
  if x.AwayTeam == team:
    return x.AST

def corners(x, team):
  if x.HomeTeam == team:
    return x.HC
  if x.AwayTeam == team:
    return x.AC

def goals_sc(x, team):
  if x.HomeTeam == team:
    return x.FTHG
  if x.AwayTeam == team:
    return x.FTAG

def goals_con(x, team):
  if x.HomeTeam == team:
    return x.FTAG
  if x.AwayTeam == team:
    return x.FTHG

In [5]:
def clean_data(epl):
  df = epl[['HomeTeam', 'AwayTeam', 'FTR']]
  for team in epl.HomeTeam.unique():
    team_data = pd.concat([epl[epl.HomeTeam == team], epl[epl.AwayTeam == team]]).sort_index().reset_index()
    team_data['Points'] = team_data.apply(lambda x: form_guide(x, team), axis =1)
    team_data['Form'] = team_data.apply(lambda x: sum(team_data['Points'][x.name-4:x.name]), axis=1)
    team_data['Total_Points'] = team_data['Points'].shift().cumsum()

    team_data['Shots'] = team_data.apply(lambda x: shots(x, team), axis=1).shift().cumsum()
    team_data['Shots_On_Target'] = team_data.apply(lambda x: shots_ot(x, team), axis=1).shift().cumsum()

    team_data['Corners'] = team_data.apply(lambda x: corners(x, team), axis=1).shift().cumsum()

    team_data['Goals_Scored'] = team_data.apply(lambda x: goals_sc(x, team), axis=1).shift().cumsum()

    team_data['Goals_Conceded'] = team_data.apply(lambda x: goals_con(x, team), axis=1).shift().cumsum()
    team_data.fillna(0, inplace=True)

    home = team_data.groupby('HomeTeam').get_group(team).set_index('index')
    away = team_data.groupby('AwayTeam').get_group(team).set_index('index')

    df.loc[home.index, 'HF'] = home['Form']
    df.loc[away.index, 'AF'] = away['Form']

    df.loc[home.index, 'HP'] = home['Total_Points']
    df.loc[away.index, 'AP'] = away['Total_Points']

    df.loc[home.index, 'HGS'] = home['Goals_Scored']
    df.loc[away.index, 'AGS'] = away['Goals_Scored']

    df.loc[home.index, 'HS'] = home['Shots']
    df.loc[away.index, 'AS'] = away['Shots']

    df.loc[home.index, 'HST'] = home['Shots_On_Target']
    df.loc[away.index, 'AST'] = away['Shots_On_Target']

    df.loc[home.index, 'HGC'] = home['Goals_Conceded']
    df.loc[away.index, 'AGC'] = away['Goals_Conceded']

    df.loc[home.index, 'HC'] = home['Corners']
    df.loc[away.index, 'AC'] = away['Corners']

  return df

In [6]:
df1 = clean_data(epl20[usable])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [8]:
numerical_cols = [cname for cname in df1.columns if df1[cname].dtype in ['Int64', 'float64']]

scal = StandardScaler()
scal.fit(df1[numerical_cols])
df1[numerical_cols] = scal.transform(df1[numerical_cols])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """Entry point for launching an IPython kernel.


In [9]:
enc = OrdinalEncoder()
enc.fit(df1[['HomeTeam']])
enc.categories_

[array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brighton', 'Burnley',
        'Chelsea', 'Crystal Palace', 'Everton', 'Leicester', 'Liverpool',
        'Man City', 'Man United', 'Newcastle', 'Norwich',
        'Sheffield United', 'Southampton', 'Tottenham', 'Watford',
        'West Ham', 'Wolves'], dtype=object)]

In [10]:
df1.HomeTeam = enc.transform(df1[['HomeTeam']])
df1.AwayTeam = enc.transform(df1[['AwayTeam']])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- AwayTeam
Feature names seen at fit time, yet now missing:
- HomeTeam

  warnings.warn(message, FutureWarning)


In [11]:
enc_ftr = OrdinalEncoder()
enc_ftr.fit(df1[['FTR']])
enc_ftr.categories_

[array(['A', 'D', 'H'], dtype=object)]

In [12]:
df1['FTR'] = enc_ftr.transform(df1[['FTR']])

In [13]:
X = np.asarray(df1.drop(['FTR'], axis=1)).astype(np.float32)
y = np.asarray(df1.FTR).astype(np.float32)

In [14]:
X_train, X_valid, y_train, y_valid = X[30:], X[:30], y[30:], y[:30] #train_test_split(X, y, test_size=0.2)

In [15]:
model = keras.Sequential([
                          keras.layers.Input(shape=X_train.shape[1:]),
                          keras.layers.Dense(300, activation='relu'),
                          keras.layers.Dense(100, activation='relu'),
                          keras.layers.Dense(50, activation='relu'),
                          keras.layers.Dense(3, activation='softmax')                          
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

hist = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))


(None, 16) <dtype: 'float32'>
(None, 3) <dtype: 'float32'>
dense (None, 16) float32
dense_1 (None, 300) float32
dense_2 (None, 100) float32
dense_3 (None, 50) float32
Epoch 1/20
11/11 [==============================] - 1s 22ms/step - loss: 1.1168 - accuracy: 0.3943 - val_loss: 1.1107 - val_accuracy: 0.3000
Epoch 2/20
11/11 [==============================] - 0s 5ms/step - loss: 1.0286 - accuracy: 0.4886 - val_loss: 1.1352 - val_accuracy: 0.3333
Epoch 3/20
11/11 [==============================] - 0s 5ms/step - loss: 1.0103 - accuracy: 0.5143 - val_loss: 1.1440 - val_accuracy: 0.3667
Epoch 4/20
11/11 [==============================] - 0s 5ms/step - loss: 0.9923 - accuracy: 0.5114 - val_loss: 1.0845 - val_accuracy: 0.4667
Epoch 5/20
11/11 [==============================] - 0s 4ms/step - loss: 0.9777 - accuracy: 0.5143 - val_loss: 1.1822 - val_accuracy: 0.4000
Epoch 6/20
11/11 [==============================] - 0s 4ms/step - loss: 0.9846 - accuracy: 0.5086 - val_loss: 1.2635 - val_accuracy:

USING GRID SEARCH TO FINE THE BEST HYPER PARAMETERS

In [16]:
def create_model(activation='relu', optimizer='adam'):
  # create model
  model = keras.models.Sequential()
  model.add(keras.layers.Input(shape=X_train.shape[1:]))
  model.add(keras.layers.Dense(300, activation=activation))
  model.add(keras.layers.Dense(100, activation=activation))
  model.add(keras.layers.Dense(50, activation='relu'))
  model.add(keras.layers.Dense(3, activation='softmax'))

  # Compile model
  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model
search_model = KerasClassifier(build_fn=create_model, epochs=20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  del sys.path[0]


In [17]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']

param_grid = dict(optimizer=optimizer, activation=activation)

grid = GridSearchCV(estimator=search_model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)



Epoch 1/20
12/12 [==============================] - 0s 3ms/step - loss: 1.1204 - accuracy: 0.4079
Epoch 2/20
12/12 [==============================] - 0s 3ms/step - loss: 1.0719 - accuracy: 0.4526
Epoch 3/20
12/12 [==============================] - 0s 3ms/step - loss: 1.0627 - accuracy: 0.4526
Epoch 4/20
12/12 [==============================] - 0s 3ms/step - loss: 1.0530 - accuracy: 0.4526
Epoch 5/20
12/12 [==============================] - 0s 3ms/step - loss: 1.0650 - accuracy: 0.4526
Epoch 6/20
12/12 [==============================] - 0s 3ms/step - loss: 1.0566 - accuracy: 0.4553
Epoch 7/20
12/12 [==============================] - 0s 3ms/step - loss: 1.0491 - accuracy: 0.4684
Epoch 8/20
12/12 [==============================] - 0s 3ms/step - loss: 1.0479 - accuracy: 0.4658
Epoch 9/20
12/12 [==============================] - 0s 3ms/step - loss: 1.0326 - accuracy: 0.4579
Epoch 10/20
12/12 [==============================] - 0s 3ms/step - loss: 1.0285 - accuracy: 0.4632
Epoch 11/20
12/12 [

In [18]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.487043 using {'activation': 'hard_sigmoid', 'optimizer': 'Adam'}
0.452756 (0.033561) with: {'activation': 'softmax', 'optimizer': 'SGD'}
0.452756 (0.033561) with: {'activation': 'softmax', 'optimizer': 'RMSprop'}
0.452756 (0.033561) with: {'activation': 'softmax', 'optimizer': 'Adagrad'}
0.305233 (0.008798) with: {'activation': 'softmax', 'optimizer': 'Adadelta'}
0.452756 (0.033561) with: {'activation': 'softmax', 'optimizer': 'Adam'}
0.452756 (0.033561) with: {'activation': 'softmax', 'optimizer': 'Adamax'}
0.452756 (0.033561) with: {'activation': 'softmax', 'optimizer': 'Nadam'}
0.413407 (0.086338) with: {'activation': 'softplus', 'optimizer': 'SGD'}
0.447611 (0.079145) with: {'activation': 'softplus', 'optimizer': 'RMSprop'}
0.458130 (0.063360) with: {'activation': 'softplus', 'optimizer': 'Adagrad'}
0.328855 (0.074973) with: {'activation': 'softplus', 'optimizer': 'Adadelta'}
0.397450 (0.041649) with: {'activation': 'softplus', 'optimizer': 'Adam'}
0.452798 (0.053163) with:

In [19]:
hist_search = grid.best_estimator_.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
11/11 [==============================] - 1s 18ms/step - loss: 1.1202 - accuracy: 0.3971 - val_loss: 1.1148 - val_accuracy: 0.3333
Epoch 2/20
11/11 [==============================] - 0s 5ms/step - loss: 1.0655 - accuracy: 0.4629 - val_loss: 1.1435 - val_accuracy: 0.3333
Epoch 3/20
11/11 [==============================] - 0s 5ms/step - loss: 1.0693 - accuracy: 0.4629 - val_loss: 1.1320 - val_accuracy: 0.3333
Epoch 4/20
11/11 [==============================] - 0s 4ms/step - loss: 1.0591 - accuracy: 0.4629 - val_loss: 1.1226 - val_accuracy: 0.3333
Epoch 5/20
11/11 [==============================] - 0s 4ms/step - loss: 1.0614 - accuracy: 0.4629 - val_loss: 1.1376 - val_accuracy: 0.3333
Epoch 6/20
11/11 [==============================] - 0s 5ms/step - loss: 1.0607 - accuracy: 0.4686 - val_loss: 1.1080 - val_accuracy: 0.3333
Epoch 7/20
11/11 [==============================] - 0s 4ms/step - loss: 1.0461 - accuracy: 0.4657 - val_loss: 1.1605 - val_accuracy: 0.3333
Epoch 8/20
11/11 [=

In [20]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [21]:
rfc = RandomForestClassifier(n_estimators=50)
xgbc = XGBClassifier(max_depth=5)
svc = SVC(C=0.1)
acc = cross_val_score(rfc, X, y, cv=5)
acc2 = cross_val_score(xgbc, X, y, cv=5)
acc3 = cross_val_score(svc, X, y, cv=5)
print('Accuracy score for Random Forest Classifier, XGB Classifier, Support Vector Machines, respectively:\n', acc.mean(), acc2.mean(), acc3.mean())

Accuracy score for Random Forest Classifier, XGB Classifier, Support Vector Machines, respectively:
 0.4184210526315789 0.45 0.4526315789473684


TRAINING USING 2021 DATA

In [22]:
df2 = clean_data(epl21[usable])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [23]:
numerical_cols = [cname for cname in df2.columns if df2[cname].dtype in ['Int64', 'float64']]

df2[numerical_cols] = scal.transform(df2[numerical_cols])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """Entry point for launching an IPython kernel.


In [24]:
df2['FTR'] = enc_ftr.fit_transform(df2[['FTR']])

In [25]:
cat = enc.categories_[0]
cat = np.append(cat, list(set(df2.HomeTeam.unique())-set(cat)))

enc.categories_ = [cat]

In [26]:
df2['HomeTeam'] = enc.transform(df2[['HomeTeam']])
df2.AwayTeam = enc.transform(df2[['AwayTeam']])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- AwayTeam
Feature names seen at fit time, yet now missing:
- HomeTeam

  warnings.warn(message, FutureWarning)


In [27]:
df2

,HomeTeam,AwayTeam,FTR,HF,AF,HP,AP,HGS,AGS,HS,AS,HST,AST,HGC,AGC,HC,AC
0,20.0,0.0,0.0,-1.476994,-1.589112,-1.392658,-1.384595,-1.420253,-1.421259,-1.589575,-1.599375,-1.534867,-1.534737,-1.561275,-1.564071,-1.592491,-1.606730
1,6.0,15.0,2.0,-1.476994,-1.589112,-1.392658,-1.384595,-1.420253,-1.421259,-1.589575,-1.599375,-1.534867,-1.534737,-1.561275,-1.564071,-1.592491,-1.606730
2,9.0,22.0,2.0,-1.476994,-1.589112,-1.392658,-1.384595,-1.420253,-1.421259,-1.589575,-1.599375,-1.534867,-1.534737,-1.561275,-1.564071,-1.592491,-1.606730
3,18.0,12.0,0.0,-1.476994,-1.589112,-1.392658,-1.384595,-1.420253,-1.421259,-1.589575,-1.599375,-1.534867,-1.534737,-1.561275,-1.564071,-1.592491,-1.606730
4,21.0,8.0,0.0,-1.476994,-1.589112,-1.392658,-1.384595,-1.420253,-1.421259,-1.589575,-1.599375,-1.534867,-1.534737,-1.561275,-1.564071,-1.592491,-1.606730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,9.0,6.0,2.0,2.277238,0.261335,2.248695,0.993466,2.258733,0.845887,2.436619,0.757489,2.433203,0.868534,0.989442,2.367527,2.318142,0.789116
376,10.0,7.0,2.0,0.400122,0.569743,3.186620,1.804169,2.927640,1.177665,2.395605,1.086353,2.414218,1.209155,0.382128,1.077471,2.222760,0.980783
377,14.0,4.0,2.0,-0.538436,-0.663888,-0.289218,0.723232,-0.361151,0.403517,0.549981,0.963029,0.287788,0.868534,2.264800,1.753215,0.998701,0.996756
378,18.0,15.0,2.0,0.712975,0.261335,2.028007,0.939420,1.868538,1.177665,1.513807,1.182272,1.464918,1.511930,1.293098,2.428958,1.046391,1.188423


In [28]:
X1 = np.asarray(df2.drop(['FTR'], axis=1)).astype(np.float32)
y1 = np.asarray(df2.FTR).astype(np.float32)  

In [29]:
X_train1, X_valid1, y_train1, y_valid1 = train_test_split(X1, y1, test_size=0.2)

In [30]:
#model.add(keras.layers.Dropout(0.2))
hist1 = model.fit(X_train1, y_train1, epochs=20, validation_data=(X_valid1, y_valid1))

Epoch 1/20
10/10 [==============================] - 0s 10ms/step - loss: 1.1073 - accuracy: 0.4605 - val_loss: 1.3192 - val_accuracy: 0.3684
Epoch 2/20
10/10 [==============================] - 0s 5ms/step - loss: 1.0271 - accuracy: 0.4967 - val_loss: 1.1280 - val_accuracy: 0.3816
Epoch 3/20
10/10 [==============================] - 0s 5ms/step - loss: 0.9914 - accuracy: 0.5197 - val_loss: 1.1551 - val_accuracy: 0.4211
Epoch 4/20
10/10 [==============================] - 0s 5ms/step - loss: 0.9605 - accuracy: 0.5197 - val_loss: 1.1237 - val_accuracy: 0.4474
Epoch 5/20
10/10 [==============================] - 0s 5ms/step - loss: 0.9483 - accuracy: 0.5461 - val_loss: 1.1239 - val_accuracy: 0.4211
Epoch 6/20
10/10 [==============================] - 0s 7ms/step - loss: 0.9193 - accuracy: 0.5395 - val_loss: 1.1700 - val_accuracy: 0.4342
Epoch 7/20
10/10 [==============================] - 0s 7ms/step - loss: 0.9218 - accuracy: 0.5395 - val_loss: 1.1105 - val_accuracy: 0.4211
Epoch 8/20
10/10 [=

TESTING ON THIS SEASON MATCHES

In [31]:
df3 = clean_data(epl22[usable])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [32]:
df3

,HomeTeam,AwayTeam,FTR,HF,AF,HP,AP,HGS,AGS,HS,AS,HST,AST,HGC,AGC,HC,AC
0,Brentford,Arsenal,H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Man United,Leeds,H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Burnley,Brighton,A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Chelsea,Crystal Palace,H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Everton,Southampton,H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,Tottenham,Crystal Palace,H,10.0,4.0,26.0,20.0,18.0,24.0,162.0,191.0,62.0,62.0,19.0,24.0,85.0,75.0
172,West Ham,Southampton,A,5.0,3.0,28.0,17.0,28.0,16.0,217.0,216.0,80.0,67.0,21.0,26.0,88.0,104.0
173,Aston Villa,Chelsea,A,6.0,5.0,22.0,38.0,23.0,39.0,191.0,284.0,60.0,102.0,25.0,12.0,88.0,118.0
174,Brighton,Brentford,H,3.0,7.0,20.0,20.0,14.0,21.0,191.0,180.0,61.0,64.0,17.0,22.0,83.0,61.0


In [33]:
numerical_cols = [cname for cname in df3.columns if df3[cname].dtype in ['Int64', 'float64']]

df3[numerical_cols] = scal.transform(df3[numerical_cols])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """Entry point for launching an IPython kernel.


In [34]:
df3['FTR'] = enc_ftr.fit_transform(df3[['FTR']])

In [35]:
cat = enc.categories_[0]
cat = np.append(cat, list(set(df3.HomeTeam.unique())-set(cat)))

enc.categories_ = [cat]

In [36]:
df3['HomeTeam'] = enc.transform(df3[['HomeTeam']])
df3.AwayTeam = enc.transform(df3[['AwayTeam']])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- AwayTeam
Feature names seen at fit time, yet now missing:
- HomeTeam

  warnings.warn(message, FutureWarning)


In [37]:
X2 = np.asarray(df3.drop(['FTR'], axis=1)).astype(np.float32)
y2 = np.asarray(df3.FTR).astype(np.float32)  

In [38]:
hist2 = model.fit(X2, y2, epochs=20)

Epoch 1/20
6/6 [==============================] - 0s 4ms/step - loss: 1.1510 - accuracy: 0.4261
Epoch 2/20
6/6 [==============================] - 0s 3ms/step - loss: 1.0495 - accuracy: 0.4773
Epoch 3/20
6/6 [==============================] - 0s 3ms/step - loss: 1.0100 - accuracy: 0.4943
Epoch 4/20
6/6 [==============================] - 0s 3ms/step - loss: 0.9667 - accuracy: 0.5057
Epoch 5/20
6/6 [==============================] - 0s 3ms/step - loss: 0.9456 - accuracy: 0.5170
Epoch 6/20
6/6 [==============================] - 0s 3ms/step - loss: 0.9210 - accuracy: 0.5284
Epoch 7/20
6/6 [==============================] - 0s 3ms/step - loss: 0.9008 - accuracy: 0.5682
Epoch 8/20
6/6 [==============================] - 0s 3ms/step - loss: 0.8852 - accuracy: 0.5625
Epoch 9/20
6/6 [==============================] - 0s 3ms/step - loss: 0.8735 - accuracy: 0.6080
Epoch 10/20
6/6 [==============================] - 0s 3ms/step - loss: 0.8673 - accuracy: 0.6136
Epoch 11/20
6/6 [======================

In [39]:
def get_stats(ht, at, df_in=epl22[usable]): #ht, at for hometeam and awayteam respectively
  df = pd.DataFrame([{'HomeTeam':ht, 'AwayTeam':at}])
  epl = df_in
  stats_dict = {}
  for team in [ht, at]:
    team_data = pd.concat([epl[epl.HomeTeam == team], epl[epl.AwayTeam == team]]).sort_index().reset_index()
    team_stats = {}
    team_data['Points'] = team_data.apply(lambda x: form_guide(x, team), axis =1)
    team_stats['Form'] = team_data['Points'].tail(4).sum()
    team_stats['Total_Points'] = team_data['Points'].sum()

    team_stats['Shots'] = team_data.apply(lambda x: shots(x, team), axis=1).sum()
    team_stats['Shots_On_Target'] = team_data.apply(lambda x: shots_ot(x, team), axis=1).sum()

    team_stats['Corners'] = team_data.apply(lambda x: corners(x, team), axis=1).sum()

    team_stats['Goals_Scored'] = team_data.apply(lambda x: goals_sc(x, team), axis=1).sum()

    team_stats['Goals_Conceded'] = team_data.apply(lambda x: goals_con(x, team), axis=1).sum()
    stats_dict[team] = team_stats
  
  for team, team_stats in stats_dict.items():
    if team == ht:
      home = team_stats
    elif team == at:
      away = team_stats

  df['HF'] = home['Form']
  df['AF'] = away['Form']

  df['HP'] = home['Total_Points']
  df['AP'] = away['Total_Points']

  df['HGS'] = home['Goals_Scored']
  df['AGS'] = away['Goals_Scored']

  df['HS'] = home['Shots']
  df['AS'] = away['Shots']

  df['HST'] = home['Shots_On_Target']
  df['AST'] = away['Shots_On_Target']

  df['HGC'] = home['Goals_Conceded']
  df['AGC'] = away['Goals_Conceded']

  df['HC'] = home['Corners']
  df['AC'] = away['Corners']

  return df

In [82]:
def predict_match(ht, at):
  df_temp = get_stats(ht, at)
  numerical_cols = [cname for cname in df_temp.columns if df_temp[cname].dtype in ['Int64', 'float64']]
  df_temp[numerical_cols] = scal.transform(df_temp[numerical_cols])
  df_temp['HomeTeam'] = enc.transform(df_temp[['HomeTeam']])
  df_temp.AwayTeam = enc.transform(df_temp[['AwayTeam']])
  array = np.asarray(df_temp).astype(np.float32)
  y_proba = model.predict(array)
  result = enc_ftr.inverse_transform([y_proba.argmax(axis=1)])
  return (result[0][0], y_proba)

In [84]:
predict_match('Norwich', 'Arsenal')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- AwayTeam
Feature names seen at fit time, yet now missing:
- HomeTeam

  warnings.warn(message, FutureWarning)


('A', array([[0.6007165 , 0.09128468, 0.30799887]], dtype=float32))

In [42]:
enc.categories_

[array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brighton', 'Burnley',
        'Chelsea', 'Crystal Palace', 'Everton', 'Leicester', 'Liverpool',
        'Man City', 'Man United', 'Newcastle', 'Norwich',
        'Sheffield United', 'Southampton', 'Tottenham', 'Watford',
        'West Ham', 'Wolves', 'Fulham', 'West Brom', 'Leeds', 'Brentford'],
       dtype=object)]

In [85]:
import joblib

In [89]:
joblib.dump(enc, 'encoder_for_teams')

['encoder_for_teams']

In [95]:
keras.models.save_model(model, 'my_epl2022_model')

INFO:tensorflow:Assets written to: my_epl2022_model/assets
